# Next Word Prediction Model using Python

Importing the necessary Python libraries and the dataset:

In [24]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Read the text file
with open('/Users/mrunalipatil/Downloads/sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    text = file.read()

Let’s tokenize the text to create a sequence of words:

In [25]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

The text is divided into individual words or tokens through tokenization. A Tokenizer object is created to perform this task. The fit_on_texts method builds a vocabulary of unique words from the text, each assigned a numerical index. The total_words variable then counts these unique words, indicating the size of the vocabulary.

Splitting the text into sequences of tokens and forming n-grams from the sequences:

In [16]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

The text is split into lines using '\n' as a delimiter. Each line is converted to a sequence of numerical tokens using the texts_to_sequences method. A for loop generates n-grams (subsequences) from these tokens, with each n-gram's last token as the target word. These n-grams are collected in the input_sequences list for training a word prediction model.

Let’s pad the input sequences to have equal length:

In [17]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

The pad_sequences function adjusts all input sequences to the same length, determined by max_sequence_len, which is the length of the longest sequence. It pads shorter sequences at the beginning (padding=pre) to match this maximum length. The resulting sequences are then converted into a numpy array for uniformity and ease of processing.

Let’s split the sequences into input and output:

In [18]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

Converting the output array into a suitable format for training a model, where each target word is represented as a binary vector.



In [19]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

Building a neural network architecture to train the model:

In [20]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2208800 (8.43 MB)
Trainable params: 2208800 (8.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


Compile and train the model:

In [21]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
3010/3010 [==============================] - 42s 14ms/step - loss: 6.2209 - accuracy: 0.0777
Epoch 2/100
3010/3010 [==============================] - 43s 14ms/step - loss: 5.5023 - accuracy: 0.1248
Epoch 3/100
3010/3010 [==============================] - 43s 14ms/step - loss: 5.1083 - accuracy: 0.1476
Epoch 4/100
3010/3010 [==============================] - 43s 14ms/step - loss: 4.7712 - accuracy: 0.1670
Epoch 5/100
3010/3010 [==============================] - 44s 14ms/step - loss: 4.4634 - accuracy: 0.1849
Epoch 6/100
3010/3010 [==============================] - 44s 15ms/step - loss: 4.1779 - accuracy: 0.2051
Epoch 7/100
3010/3010 [==============================] - 44s 14ms/step - loss: 3.9053 - accuracy: 0.2320
Epoch 8/100
3010/3010 [==============================] - 43s 14ms/step - loss: 3.6489 - accuracy: 0.2620
Epoch 9/100
3010/3010 [==============================] - 43s 14ms/step - loss: 3.4045 - accuracy: 0.2961
Epoch 10/100
3010/3010 [==============================]

The ‘metrics’ parameter is set to ‘accuracy’ to monitor the accuracy during training. After compiling the model, the ‘fit’ method is called to train the model on the input sequences ‘X’ and the corresponding output ‘y’. The ‘epochs’ parameter specifies the number of times the training process will iterate over the entire dataset. The ‘verbose’ parameter is set to ‘1’ to display the training process.

Now Let's generate the next word predictions using our model:

In [22]:
seed_text = "I will leave if they"
next_words = 3

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 7ms/step
I will leave if they may have been
